Clean up our environment

In [1]:
USE master;
GO
DECLARE @dbname NVARCHAR(128) = N'UnitTesting';

IF (
    EXISTS(
        SELECT name
        FROM master.dbo.sysdatabases
        WHERE ('[' + name + ']' = @dbname OR name = @dbname)
    )
)
BEGIN
    ALTER DATABASE [UnitTesting] SET SINGLE_USER WITH ROLLBACK IMMEDIATE;
END
GO
DROP DATABASE IF EXISTS UnitTesting;
GO
-- Cleanup
DROP PROCEDURE IF EXISTS [dbo].[Person_Create];
DROP PROCEDURE IF EXISTS [dbo].[Person_GetAll];
GO

Create the database

In [3]:
CREATE DATABASE UnitTesting;
GO

Create the tables

In [4]:
USE [UnitTesting];
GO

CREATE TABLE [dbo].[Person]
(
    [PersonId] [INT] IDENTITY(1, 1) NOT NULL,
    [FirstName] [VARCHAR](50) NOT NULL,
    [LastName] [VARCHAR](50) NOT NULL,
    [Address] [VARCHAR](100) NOT NULL,
    [City] [VARCHAR](50) NOT NULL,
    [Zipcode] [VARCHAR](8) NOT NULL,
    [Country] [VARCHAR](50) NOT NULL,
    [Email] [VARCHAR](100) NOT NULL,
    CONSTRAINT [PK_Person]
        PRIMARY KEY CLUSTERED ([PersonId] ASC)
        WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON,
              ALLOW_PAGE_LOCKS = ON
             ) ON [PRIMARY]
) ON [PRIMARY];
GO

CREATE TABLE [dbo].[Customer]
(
    [CustomerID] [INT] IDENTITY(1, 1) NOT NULL,
    [PersonID] [INT] NOT NULL,
    [AccountNumber] [INT] NOT NULL,
    CONSTRAINT [PK_Customer]
        PRIMARY KEY CLUSTERED ([CustomerID] ASC)
        WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON,
              ALLOW_PAGE_LOCKS = ON
             ) ON [PRIMARY]
) ON [PRIMARY];
GO

ALTER TABLE [dbo].[Customer] WITH CHECK
ADD CONSTRAINT [FK_Customer_person]
    FOREIGN KEY ([PersonID])
    REFERENCES [dbo].[Person] ([PersonId]);
GO

ALTER TABLE [dbo].[Customer] CHECK CONSTRAINT [FK_Customer_person];
GO

Create the procedures

In [4]:
USE [UnitTesting];
GO

/*
Description:
Create a new person

Changes:
Date		Who						Notes
----------	---						--------------------------------------------------------------
10/08/2019	Sander Stad				Initial procedure
*/
CREATE PROCEDURE [dbo].[Person_Create]
    @PersonID INT OUTPUT,
    @FirstName VARCHAR(50) = NULL,
    @LastName VARCHAR(50) = NULL,
    @Address VARCHAR(100) = NULL,
    @City VARCHAR(50) = NULL,
    @Zipcode VARCHAR(8) = NULL,
    @Country VARCHAR(50) = NULL,
    @Email VARCHAR(100) = NULL
AS
BEGIN


    -- Set session options to make sure transactions are aborted correctly
    -- and the procedure doesn't return the count
    SET XACT_ABORT, NOCOUNT ON;

    -- Check the parameters
    IF (@FirstName IS NULL)
    BEGIN
        ;
        THROW 50000, 'Invalid parameter: @FirstName cannot be NULL!', 1;
        RETURN;
    END;

    -- Declare variables
    DECLARE @sqlcmd NVARCHAR(MAX);
    DECLARE @params NVARCHAR(MAX);



    -- Set the SQL command
    SET @sqlcmd
        = N'
			INSERT INTO dbo.Person(FirstName,LastName,Address,City,Zipcode,Country,Email) 
			VALUES (@FirstName, @LastName, @Address, @City, @Zipcode, @Country, @Email);

			SELECT @PersonID = SCOPE_IDENTITY();
		';

    SET @params
        = N'
			@PersonID INT OUTPUT,
			@FirstName VARCHAR(50),
			@LastName VARCHAR(50),
			@Address VARCHAR(100),
			@City VARCHAR(50),
			@Zipcode VARCHAR(8),
			@Country VARCHAR(50),
			@Email VARCHAR(100)
		';

    EXECUTE sp_executesql @stmnt = @sqlcmd,
                          @params = @params,
                          @FirstName = @FirstName,
                          @LastName = @LastName,
                          @Address = @Address,
                          @City = @City,
                          @Zipcode = @Zipcode,
                          @Country = @Country,
                          @Email = @Email,
                          @PersonID = @PersonID OUTPUT;

END;

GO

/*
Description:
Procedure to get all persons

Changes:
Date		Who						Notes
----------	---						--------------------------------------------------------------
10/08/2019	Sander Stad				Initial procedure
*/
CREATE PROCEDURE [dbo].[Person_GetAll]
AS
BEGIN
    SET NOCOUNT ON;

    -- Execute the SQL command
    SELECT PersonId,
           FirstName,
           LastName,
           Address,
           City,
           Zipcode,
           Country,
           Email
    FROM [dbo].[Person];

END;